# 12. Seq2seq으로 번역기 만들기 [프로젝트]
프로젝트: 한영 번역기 만들기

## 0. 준비

In [1]:
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import re
from konlpy.tag import Mecab
import numpy as np
import matplotlib.ticker as ticker


import warnings

warnings.filterwarnings(action='ignore') 

In [2]:
with open('./data/korean-english-park.train.ko', 'r') as f:
    train_ko_data = f.read().splitlines()

with open('./data/korean-english-park.train.en', 'r') as f:
    train_en_data = f.read().splitlines()

In [3]:
print('train_ko_data : ', len(train_ko_data))
print('train_en_data : ', len(train_en_data))

train_ko_data :  94123
train_en_data :  94123


In [4]:
for i in range(0,500,100):
    print('>> ', train_ko_data[i])
    print('>> ', train_en_data[i], '\n')

>>  개인용 컴퓨터 사용의 상당 부분은 "이것보다 뛰어날 수 있느냐?"
>>  Much of personal computing is about "can you top this?" 

>>  제 23차 연례 컴덱스 박람회의 개회사를 한 케이츠는 2년여전 기술 산업의 거품이 붕괴된 이후에 첨단 기술에 대해 부정적인 인식이 있다고 말했다.
>>  Gates, who opened the 23rd annual Comdex trade show, said there was a negative perception of high tech following the collapse of the tech bubble about two years ago. 

>>  국제 원자력 기구는 북한이 핵 무기 개발 계획을 중지하고, 즉각적으로 "모든 관련 시설들"을 공개하여 사찰을 받으라고 요구했다.
>>  The International Atomic Energy Agency called on North Korea to end any nuclear weapon program and open "all relevant facilities" to inspections immediately. 

>>  일본에서 124명의 사망자를 낸 폐암 치료제
>>  Microsoft had "leveraged its PC monopoly in which it is unfairly advantaged." 

>>  그러나 미국은 개개의 시험과 성인의 식자율(識字率)에서 하위에 머물렀다.
>>  The United States, however, finished low in each test and in adult literacy. 



## Step 2. 데이터 정제

In [5]:
# set을 이용한 중복 제거
cleaned_corpus = list(set(zip(train_ko_data, train_en_data)))

In [6]:
len(cleaned_corpus)

78968

In [7]:
for i in range(0,500,100):
    print('>> ', cleaned_corpus[i][0])
    print('>> ', cleaned_corpus[i][1], '\n')

>>  모든 혐의들이 인정될 경우 러시아 언론에 의해 ‘비트세브스키 미치광이’로 불리는 피추슈킨은 지난 1992년 52명을 살해한 안드레이 치카틸로 이후 최악의 연쇄살인범으로 기록될 전망이다.
>>  If convicted, Pichushkin called the "Bitsevsky Maniac" by Russian media after the Moscow park where many of the alleged victims were killed would be Russia's most deadly serial killer since Andrei Chikatilo, convicted in 1992 of 52 murders. 

>>  그린피스와 다른 환경단체는 중국의 환경 정책을 비판해 왔다.
>>  Greenpeace and other environmental activists have criticized China for its environmental policies. 

>>  그는 선행을 할 것, 힘과 더불어 두뇌에 의존할 것, 공평하게 통치할 것, 계획을 잘 세울 것, 사람들의 비밀을 지키고 타인의 실수에서 교훈을 얻을 것을 요구한다.
>>  He calls for doing good, depending on brains as well as *brawn, ruling fairly, planning well, keeping people’s secrets and learning from others’ mistakes. 

>>  이어 “진심으로 대화하고 마음 깊이 변화하면 모든 악의 원인을 이겨 낼 수 있다”고 덧붙였다.
>>  "Only through the conversion of hearts, only through a change in the depths of our hearts can the cause of all this evil be overcome." 

>>  한편 누르 하산 후세인 소말리아 총리는 이번 교전사태에 대해 공식 언급을 피했

In [8]:
# 한국어 전처리
# 정규식 변경, mecab 사용
def preprocess_sentence_ko(sentence):
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = re.sub(r"[^가-힣?.!,]+", " ", sentence)

    sentence = sentence.strip()
    
    mecab = Mecab()
    sentence = mecab.morphs(sentence)

    return sentence

In [9]:
# 영어 전처리
def preprocess_sentence_en(sentence, s_token=False, e_token=False):
    sentence = sentence.lower().strip()

    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = re.sub(r"[^a-zA-Z?.!,]+", " ", sentence)

    sentence = sentence.strip()

    if s_token:
        sentence = '<start> ' + sentence

    if e_token:
        sentence += ' <end>'
    
    sentence = sentence.split()
    
    return sentence

In [10]:
# 토큰 길이가 40 이하인 데이터 선별
kor_corpus = []
eng_corpus = []

for x in cleaned_corpus:
    ko = preprocess_sentence_ko(x[0])
    en = preprocess_sentence_en(x[1], s_token=True, e_token=True)
    
    if len(ko) <= 40 and len(en) <= 40:
        kor_corpus.append(ko)
        eng_corpus.append(en)

In [11]:
print(len(kor_corpus), len(eng_corpus))

for i in range(0,500,100):
    print('>> ', kor_corpus[i])
    print('>> ', eng_corpus[i], '\n')

63139 63139
>>  ['레바논', '에서', '의', '폭발', '에', '유엔', '병사', '들', '사망', '.']
>>  ['<start>', 'authorities', 'have', 'said', 'that', 'fatah', 'al', 'islam', 'militants', 'who', 'have', 'been', 'arrested', 'and', 'interrogated', 'have', 'confessed', 'there', 'was', 'a', 'plan', 'to', 'attack', 'the', 'un', '.', '<end>'] 

>>  ['스리랑카', '당국', '은', '열차', '가', '콜롬보', '시', '중심가', '에', '있', '는', '스', '링', '랑', '카', '의', '주요', '역사', '인', '포트', '역', '에', '열차', '가', '들어설', '때', '자살', '폭탄', '테러', '가', '발생', '했', '다고', '밝혔', '다', '.']
>>  ['<start>', 'the', 'suicide', 'bombing', 'took', 'place', 'as', 'a', 'train', 'was', 'pulling', 'into', 'fort', 'station', 'the', 'country', 's', 'main', 'railway', 'station', 'in', 'colombo', 's', 'city', 'center', ',', 'authorities', 'said', '.', '<end>'] 

>>  ['게임', '은', '연령', '에', '관련', '된', '인지', '기능', '감퇴', '와', '시각', '적', '기민성', '을', '개선', '하', '도록', '고안', '됐', '다', '.']
>>  ['<start>', 'they', 're', 'designed', 'to', 'reverse', 'age', 'related', 'cognitive',

## Step 3. 데이터 토큰화

In [12]:
# 단어수 15000으로 설정, 텐서 변환
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=15000, filters='', oov_token="<unk>")
    tokenizer.fit_on_texts(corpus)
    
    tensor = tokenizer.texts_to_sequences(corpus)
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')
    
    return tensor, tokenizer

In [13]:
enc_tensor, enc_tokenizer = tokenize(kor_corpus)
dec_tensor, dec_tokenizer = tokenize(eng_corpus)

In [14]:
len(enc_tokenizer.index_word), len(enc_tokenizer.index_word)

(37077, 37077)

In [15]:
enc_tensor[0]

array([ 712,   17,    7,  389,    9,  257, 1004,   16,   90,    2,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0], dtype=int32)

In [16]:
dec_tensor[0]

array([   4,  207,   27,   13,   16, 1982,  179, 2317,  393,   40,   27,
         39,  345,   11, 8666,   27, 4284,   73,   18,    9,  328,    7,
        228,    2,  734,    3,    5,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0], dtype=int32)

## Step 4. 모델 설계

In [17]:
class BahdanauAttention(tf.keras.layers.Layer):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.w_dec = tf.keras.layers.Dense(units)
        self.w_enc = tf.keras.layers.Dense(units)
        self.w_com = tf.keras.layers.Dense(1)
    
    def call(self, h_enc, h_dec):
        # h_enc shape: [batch x length x units]
        # h_dec shape: [batch x units]

        h_enc = self.w_enc(h_enc)
        h_dec = tf.expand_dims(h_dec, 1)
        h_dec = self.w_dec(h_dec)

        score = self.w_com(tf.nn.tanh(h_dec + h_enc))
        
        attn = tf.nn.softmax(score, axis=1)

        context_vec = attn * h_enc
        context_vec = tf.reduce_sum(context_vec, axis=1)

        return context_vec, attn

In [18]:
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units):
        super(Encoder, self).__init__()

        self.enc_units = enc_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(enc_units, return_sequences=True)

    def call(self, x):
        out = self.embedding(x)
        out = self.gru(out)

        return out

In [19]:
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units):
        super(Decoder, self).__init__()
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(dec_units,
                                       return_sequences=True,
                                       return_state=True)
        self.fc = tf.keras.layers.Dense(vocab_size)

        self.attention = BahdanauAttention(self.dec_units)

    def call(self, x, h_dec, enc_out):
        context_vec, attn = self.attention(enc_out, h_dec)

        out = self.embedding(x)
        out = tf.concat([tf.expand_dims(context_vec, 1), out], axis=-1)

        out, h_dec = self.gru(out)
        out = tf.reshape(out, (-1, out.shape[2]))
        out = self.fc(out)

        return out, h_dec, attn

In [20]:
BATCH_SIZE     = 64
SRC_VOCAB_SIZE = len(enc_tokenizer.index_word) + 1 # 예: len(enc_tokenizer.index_word) + 1
TGT_VOCAB_SIZE = len(dec_tokenizer.index_word) + 1 # 예: len(dec_tokenizer.index_word) + 1

units         = 128
embedding_dim = 128

encoder = Encoder(SRC_VOCAB_SIZE, embedding_dim, units)
decoder = Decoder(TGT_VOCAB_SIZE, embedding_dim, units)

# sample input
sequence_len = 40

sample_enc = tf.random.uniform((BATCH_SIZE, sequence_len))
sample_output = encoder(sample_enc)

print ('Encoder Output:', sample_output.shape)

sample_state = tf.random.uniform((BATCH_SIZE, units))

sample_logits, h_dec, attn = decoder(tf.random.uniform((BATCH_SIZE, 1)),
                                     sample_state, sample_output)

print ('Decoder Output:', sample_logits.shape)
print ('Decoder Hidden State:', h_dec.shape)
print ('Attention:', attn.shape)

Encoder Output: (64, 40, 128)
Decoder Output: (64, 38542)
Decoder Hidden State: (64, 128)
Attention: (64, 40, 1)


In [21]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss = loss_object(real, pred)
    
    mask = tf.cast(mask, dtype=loss.dtype)
    loss *= mask
    
    return tf.reduce_mean(loss)

In [22]:
@tf.function
def train_step(src, tgt, encoder, decoder, optimizer, dec_tok):
    bsz = src.shape[0]
    loss = 0

    with tf.GradientTape() as tape:
        enc_out = encoder(src)
        h_dec = enc_out[:, -1]
        
        dec_src = tf.expand_dims([dec_tok.word_index['<start>']] * bsz, 1)

        for t in range(1, tgt.shape[1]):
            pred, h_dec, _ = decoder(dec_src, h_dec, enc_out)

            loss += loss_function(tgt[:, t], pred)
            dec_src = tf.expand_dims(tgt[:, t], 1)
        
    batch_loss = (loss / int(tgt.shape[1]))

    variables = encoder.trainable_variables + decoder.trainable_variables
    gradients = tape.gradient(loss, variables)
    optimizer.apply_gradients(zip(gradients, variables))
    
    return batch_loss

## Step 5. 훈련하기

In [23]:
from tqdm import tqdm    # tqdm
import random

EPOCHS = 30

for epoch in range(EPOCHS):
    total_loss = 0

    idx_list = list(range(0, enc_tensor.shape[0], BATCH_SIZE))
    random.shuffle(idx_list)
    t = tqdm(idx_list)

    for (batch, idx) in enumerate(t):
        batch_loss = train_step(enc_tensor[idx:idx+BATCH_SIZE],
                                dec_tensor[idx:idx+BATCH_SIZE],
                                encoder,
                                decoder,
                                optimizer,
                                dec_tokenizer)

        total_loss += batch_loss

        t.set_description_str('Epoch %2d' % (epoch + 1))
        t.set_postfix_str('Loss %.4f' % (total_loss.numpy() / (batch + 1)))

Epoch 30: 100%|██████████| 987/987 [01:35<00:00, 10.31it/s, Loss 1.8370]


In [32]:
def evaluate(sentence, encoder, decoder):
    attention = np.zeros((dec_tensor.shape[-1], enc_tensor.shape[-1]))
    
    sentence = preprocess_sentence_ko(sentence)
    inputs = enc_tokenizer.texts_to_sequences([sentence])
    inputs = tf.keras.preprocessing.sequence.pad_sequences(inputs,
                                                           maxlen=enc_tensor.shape[-1],
                                                           padding='post')

    result = ''

    enc_out = encoder(inputs)

    dec_hidden = enc_out[:, -1]
    dec_input = tf.expand_dims([dec_tokenizer.word_index['<start>']], 0)

    for t in range(dec_tensor.shape[-1]):
        predictions, dec_hidden, attention_weights = decoder(dec_input,
                                                             dec_hidden,
                                                             enc_out)

        attention_weights = tf.reshape(attention_weights, (-1, ))
        attention[t] = attention_weights.numpy()

        predicted_id = \
        tf.argmax(tf.math.softmax(predictions, axis=-1)[0]).numpy()

        result += dec_tokenizer.index_word[predicted_id] + ' '

        if dec_tokenizer.index_word[predicted_id] == '<end>':
            return result, sentence, attention

        dec_input = tf.expand_dims([predicted_id], 0)

    return result, sentence, attention

In [33]:
def plot_attention(attention, sentence, predicted_sentence):
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(1, 1, 1)
    ax.matshow(attention, cmap='viridis')

    fontdict = {'fontsize': 14}

    ax.set_xticklabels([''] + sentence, fontdict=fontdict, rotation=90)
    ax.set_yticklabels([''] + predicted_sentence, fontdict=fontdict)

    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()

In [34]:
def translate(sentence, encoder, decoder):
    result, sentence, attention = evaluate(sentence, encoder, decoder)

    print('Input: %s' % (sentence))
    print('Predicted translation: {}'.format(result))
    
    attention = attention[:len(result), :len(sentence)]

In [35]:
import matplotlib.font_manager as fm
import matplotlib as mpl

fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=9)
plt.rc('font', family='NanumBarunGothic')

In [36]:
translate("오바마는 대통령이다.", encoder, decoder)

Input: ['오바마', '는', '대통령', '이', '다', '.']
Predicted translation: obama is the president elect . <end> 


In [37]:
translate("시민들은 도시 속에 산다.", encoder, decoder)

Input: ['시민', '들', '은', '도시', '속', '에', '산다', '.']
Predicted translation: the people are not uncommon in the city . <end> 


In [38]:
translate("커피는 필요 없다.", encoder, decoder)

Input: ['커피', '는', '필요', '없', '다', '.']
Predicted translation: the <unk> is not a lot of the <unk> of the <unk> of the <unk> of the <unk> of the <unk> of the <unk> of the <unk> of the <unk> of the <unk> of the <unk> of the <unk> of 


In [39]:
translate("일곱 명의 사망자가 발생했다.", encoder, decoder)

Input: ['일곱', '명', '의', '사망자', '가', '발생', '했', '다', '.']
Predicted translation: two of the two people were killed . <end> 


## 결론

### 루브릭
|평가문항|상세기준|
|:---|:---|
|1. 번역기 모델 학습에 필요한 텍스트 데이터 전처리가 한국어 포함하여 잘 이루어졌다.	|구두점, 대소문자, 띄어쓰기, 한글 형태소분석 등 번역기 모델에 요구되는 전처리가 정상적으로 진행되었다.|
|2. Attentional Seq2seq 모델이 정상적으로 구동된다.|seq2seq 모델 훈련 과정에서 training loss가 안정적으로 떨어지면서 학습이 진행됨이 확인되었다.|
|3. 테스트 결과 의미가 통하는 수준의 번역문이 생성되었다.|테스트용 디코더 모델이 정상적으로 만들어져서, 정답과 어느 정도 유사한 영어 번역이 진행됨을 확인하였다.|

### 고찰
모델이 예측한 결과를 **네이버 파파고 번역기**의 결과와 비교하였다.

`input` 오바마는 대통령이다.  
`predict` obama is the president elect.  
`papago` Obama is the president.
> 네 가지 에문 중 가장 잘 번역하였다. 

`input` 시민들은 도시 속에 산다.  
`predict` the people are not uncommon in the city.  
`papago` Citizens live in cities.
> '도시'와 '사람들' 이라는 키워드는 들어갔으나, 문장의 내용이 맞지 않았다.

`input` 커피는 필요 없다.  
`predict` the \<unk> is not a lot of the \<unk> of the \<unk> of the \<unk> of the \<unk> of the \<unk> of the \<unk> of the \<unk> of the \<unk> of the \<unk> of the \<unk> of the \<unk> of  
`papago` I don't need coffee.
> 전처리나 모델에 문제가 있었는지 제대로 동작하지 않았다.

`input` 일곱 명의 사망자가 발생했다.  
`predict` two of the two people were killed.  
`papago` Seven people were killed.
> '사람들' 과 '죽음' 키워드는 포함되었으나, 세부적인 부분(인원 수)이 맞지 않았다.

> \>\>  일본에서 124명의 사망자를 낸 폐암 치료제  
> \>\>  Microsoft had "leveraged its PC monopoly in which it is unfairly advantaged." 

처음 데이터를 불러오고 출력했을 때 위와 같이 한글 문장과 영어 문장 쌍이 맞지 않는 경우가 있었다.

> \>\>  ['레바논', '에서', '의', '폭발', '에', '유엔', '병사', '들', '사망', '.']  
> \>\>  ['\<start\>', 'authorities', 'have', 'said', 'that', 'fatah', 'al', 'islam', 'militants', 'who', 'have', 'been', 'arrested', 'and', 'interrogated', 'have', 'confessed', 'there', 'was', 'a', 'plan', 'to', 'attack', 'the', 'un', '.', '\<end\>']   

이런 문제가 토큰화 이후에도 보인 것으로 보아, 모델 훈련에 잘못된 데이터가 섞여있었을 것이라고 추측할 수 있다. (특히 세 번째 예시 문장에서) 이상적이지 않은 결과가 나온 것에 대해, 같은 데이터와 전처리에서 트랜스포머 등 다른 모델을 사용했을 때 결과가 어떻게 바뀔지 궁금하다. 


### 회고
|KPT|내용|
|:---|:---|
|Keep|모델을 훈련시키고 예시 문장에 대한 결과를 출력하여 비교하였다.|
|Problem|원인을 찾지 못했지만 시각화 단계에서 이상하게 출력되는 문제가 있었다.|
|Try|시각화 오류를 해결해 보고, 해당 데이터셋과 모델에 대해 더 알아보면서 추가적인 전처리를 생각해본다.|